In [ ]:
from binance.client import Client
import time
import gate_api
from gate_api.exceptions import ApiException, GateApiException


# API anahtarlarınızı kullanarak bir Binance client nesnesi oluşturun
client = Client(api_key='key', api_secret='secret')



# Toplu işlemleri işleyerek toplam satış ve alış miktarlarını hesapla

while True:
    
    # PEPE/USDT için son 5 toplu işlemi sorgula
    aggregate_trades = client.get_aggregate_trades(symbol='BTCUSDT', limit=100)
    current_price = client.get_symbol_ticker(symbol="BTCUSDT")
    total_sold = 0
    total_bought = 0

    for trade in aggregate_trades:
        amount = float(trade['p']) * float(trade['q'])  # Fiyat x Miktar
        if trade['m']:  # Eğer 'm' anahtarı True ise, işlem satıcı tarafından başlatıldı ve bu bir satış işlemidir
            total_sold += amount
        else:  # Eğer 'm' anahtarı False ise, işlem alıcı tarafından başlatıldı ve bu bir alım işlemidir
            total_bought += amount

    # Toplam satış ve alış miktarlarını yazdır
    Denge = total_bought - total_sold
    print(f"Satış {total_sold:.1f}  Alış: {total_bought:.1f}, Fiyat: {current_price['price']}, Denge: {Denge: .1f}")
    #print(f"Toplamda {total_bought:.1f} USDT değerinde PEPE satın alındı.")
    
    time.sleep(6)

Satış 145373.8  Alış: 17582.9, Fiyat: 68040.02000000, Denge: -127790.9


## YOU CAN DO THIS!!

In [2]:
from binance.client import Client
import time
import gate_api
import requests
from gate_api.exceptions import ApiException, GateApiException
import numpy as np

# API anahtarları
BINANCE_API_KEY = 'key'
BINANCE_API_SECRET = 'secret'
GATEIO_API_KEY = 'key'
GATEIO_API_SECRET = 'secret'

# Sabitler
SYMBOL = 'WIF_USDT'
BINANCE_SYMBOL = 'WIFUSDT'
SLEEP_DURATION = 3 # saniye

# Binance ve Gate.io için API client nesnelerini oluşturma
client = Client(api_key=BINANCE_API_KEY, api_secret=BINANCE_API_SECRET)
configuration = gate_api.Configuration(key=GATEIO_API_KEY, secret=GATEIO_API_SECRET)
gateio_api_client = gate_api.ApiClient(configuration)
gateio_spot_api = gate_api.SpotApi(gateio_api_client)

def calculate_ema(closes, period=1):
    ema = np.zeros_like(closes)
    ema[0] = closes[0]  # Başlangıç değeri, ilk kapanış fiyatıyla aynı
    # EMA hesaplaması
    alpha = 2 / (period + 1)
    for i in range(1, len(closes)):
        ema[i] = closes[i] * alpha + ema[i - 1] * (1 - alpha)
    return ema[-1]

def calculate_ema2(closes, period=2):
    ema = np.zeros_like(closes)
    ema[0] = closes[0]  # Başlangıç değeri, ilk kapanış fiyatıyla aynı
    # EMA hesaplaması
    alpha = 2 / (period + 1)
    for i in range(1, len(closes)):
        ema[i] = closes[i] * alpha + ema[i - 1] * (1 - alpha)
    return ema[-1]

def calculate_rsi(closes, period=2):
    deltas = np.diff(closes)
    seed = deltas[:period]
    up = seed[seed >= 0].sum() / period
    down = -seed[seed < 0].sum() / period
    rs = up / down
    rsi = np.zeros_like(closes)
    rsi[:period] = 100. - 100. / (1. + rs)
    for i in range(period, len(closes)):
        delta = deltas[i - 1]  # `i` öğesiyle bir önceki kapanış arasındaki fark
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta
        up = (up * (period - 1) + upval) / period
        down = (down * (period - 1) + downval) / period
        rs = up / down
        rsi[i] = 100. - 100. / (1. + rs)
    return rsi[-1]

def get_balance_difference_gateio(api_client, spot_api, symbol):
    current_price = float(spot_api.list_tickers(currency_pair=symbol)[0].last)
    return current_price

def get_balance_difference_binance(client, symbol):
    current_price = client.get_symbol_ticker(symbol=symbol)
    return current_price['price']

SIMULATION_BALANCE_USDT = 10.0  # Başlangıç simülasyon bakiyesi (USDT)
SIMULATION_BTC = 0  # Başlangıçta BTC miktarı

try:
    while True:
        #gateio ve binance de priceları ve balanceyi hesapla!
        price_gateio= get_balance_difference_gateio(gateio_api_client, gateio_spot_api, SYMBOL)
        price_binance = get_balance_difference_binance(client, BINANCE_SYMBOL)
        
        #pricelar str geliyor floata çevirdik.
        price_gateio_f = float(price_gateio)
        price_binance_f = float(price_binance)
        
        #Borsalar arası fiyat farkından alım satım emri vermeye çalıştık, alım satım farkları her iki borsa içinde toplayıp denge_toplami diye bir değişken türettik
        fiyat_farki = price_binance_f - price_gateio_f
        #denge_toplami = balance_diff_gateio_f + balance_diff_binance_f

        # 1 dakikalık RSI hesaplama
        klines = client.get_historical_klines(BINANCE_SYMBOL, '1m', '100 minutes ago UTC')
        closes = np.array([float(kline[4]) for kline in klines])
        rsi = calculate_rsi(closes, period=2)
        
        # mum en yüksek kapanış fiyatları çekme
        latest_candle = klines[-1]
        high = float(latest_candle[2])  # En yüksek fiyat
        low = float(latest_candle[3])   # En düşük fiyat
        open_price = float(latest_candle[1])  # Açılış fiyatı
        close = float(latest_candle[4])  # Kapanış fiyatı
        #print(f"En Yüksek Fiyat: {high}, En Düşük Fiyat: {low}, Açılış Fiyatı: {open_price}, Kapanış Fiyatı: {close}")
        
        #EMA Hesaplama
        ema = calculate_ema(closes, period=1)
        ema2 = calculate_ema2(closes, period=2)

        # Tüm verilerin birleştirilerek yazdırılması
        print(f"Fiyat:{price_gateio_f:.4f}, Fark: {fiyat_farki:.4f}, RSI:{rsi:.4f}, EMA1: {ema:.4f}, EMA2: {ema2:.4f}")
        #Alım Sinyal
        if ema >= ema2 and SIMULATION_BALANCE_USDT >= 5 and rsi < 30:  
            bought_btc = 10 / price_gateio_f
            SIMULATION_BALANCE_USDT -= 10
            SIMULATION_BTC += bought_btc
            print(f"Alım yapıldı: {bought_btc} BTC, Yeni bakiye: {SIMULATION_BALANCE_USDT} USDT, BTC: {SIMULATION_BTC}")
        
        # Satım sinyali
        elif ema2 > ema and SIMULATION_BTC * price_gateio_f >= 10 and rsi > 70:
            sold_usdt = SIMULATION_BTC * price_gateio_f
            fee = sold_usdt * 0.002
            net_usdt = sold_usdt - fee
            SIMULATION_BALANCE_USDT += net_usdt
            SIMULATION_BTC = 0
            print(f"Satım yapıldı: {sold_usdt} USDT, Yeni bakiye: {SIMULATION_BALANCE_USDT} USDT, BTC: {SIMULATION_BTC}")
        
        time.sleep(SLEEP_DURATION)
        
except KeyboardInterrupt:
    print("Döngü manuel olarak durduruldu.")
except Exception as e:
    print(f"Bir hata oluştu: {e}")


Fiyat:3.6062, Fark: 0.0026, RSI:78.0285, EMA1: 3.6088, EMA2: 3.6059
Fiyat:3.6075, Fark: 0.0014, RSI:78.1743, EMA1: 3.6089, EMA2: 3.6059
Fiyat:3.6075, Fark: 0.0025, RSI:79.6592, EMA1: 3.6100, EMA2: 3.6067
Fiyat:3.6077, Fark: 0.0022, RSI:79.5326, EMA1: 3.6099, EMA2: 3.6066
Fiyat:3.6077, Fark: 0.0010, RSI:77.8808, EMA1: 3.6087, EMA2: 3.6058
Fiyat:3.6077, Fark: 0.0010, RSI:77.8808, EMA1: 3.6087, EMA2: 3.6058
Fiyat:3.6100, Fark: 0.0026, RSI:84.8221, EMA1: 3.6121, EMA2: 3.6100
Fiyat:3.6100, Fark: 0.0002, RSI:79.2746, EMA1: 3.6092, EMA2: 3.6081
Fiyat:3.6100, Fark: -0.0001, RSI:80.9549, EMA1: 3.6099, EMA2: 3.6085
Fiyat:3.6104, Fark: -0.0018, RSI:79.7842, EMA1: 3.6094, EMA2: 3.6082
Fiyat:3.6092, Fark: 0.0003, RSI:74.8599, EMA1: 3.6084, EMA2: 3.6075
Fiyat:3.6071, Fark: 0.0001, RSI:64.8053, EMA1: 3.6072, EMA2: 3.6067
Fiyat:3.6098, Fark: -0.0004, RSI:79.7842, EMA1: 3.6094, EMA2: 3.6082
Fiyat:3.6092, Fark: -0.0008, RSI:74.8599, EMA1: 3.6084, EMA2: 3.6075
Fiyat:3.6084, Fark: 0.0001, RSI:75.8405, EMA

C:\Users\selda\AppData\Local\Temp\ipykernel_11156\114789494.py:48: RuntimeWarning: divide by zero encountered in scalar divide
  rs = up / down
C:\Users\selda\AppData\Local\Temp\ipykernel_11156\114789494.py:61: RuntimeWarning: divide by zero encountered in scalar divide
  rs = up / down


Fiyat:3.6075, Fark: -0.0065, RSI:31.0713, EMA1: 3.6010, EMA2: 3.6023
Fiyat:3.6075, Fark: -0.0069, RSI:29.7923, EMA1: 3.6006, EMA2: 3.6020
Fiyat:3.6075, Fark: -0.0100, RSI:22.5866, EMA1: 3.5975, EMA2: 3.6000
Fiyat:3.6000, Fark: -0.0024, RSI:22.7642, EMA1: 3.5976, EMA2: 3.6000
Fiyat:3.5986, Fark: -0.0017, RSI:21.5766, EMA1: 3.5969, EMA2: 3.5996
Fiyat:3.5986, Fark: -0.0028, RSI:19.9417, EMA1: 3.5958, EMA2: 3.5988
Fiyat:3.5986, Fark: -0.0016, RSI:21.7386, EMA1: 3.5970, EMA2: 3.5996
Fiyat:3.5986, Fark: -0.0022, RSI:20.8014, EMA1: 3.5964, EMA2: 3.5992
Fiyat:3.5986, Fark: 0.0016, RSI:29.1915, EMA1: 3.6004, EMA2: 3.6019
Fiyat:3.6000, Fark: 0.0009, RSI:30.7414, EMA1: 3.6009, EMA2: 3.6022
Fiyat:3.5995, Fark: 0.0006, RSI:28.3343, EMA1: 3.6001, EMA2: 3.6017
Fiyat:3.5995, Fark: -0.0009, RSI:24.7070, EMA1: 3.5986, EMA2: 3.6007
Fiyat:3.5983, Fark: -0.0028, RSI:19.5379, EMA1: 3.5955, EMA2: 3.5986
Fiyat:3.5983, Fark: -0.0033, RSI:18.9001, EMA1: 3.5950, EMA2: 3.5983
Fiyat:3.5983, Fark: -0.0021, RSI:27.9

Fiyat:3.6392, Fark: 0.0012, RSI:92.3984, EMA1: 3.6402, EMA2: 3.6378
Fiyat:3.6394, Fark: 0.0007, RSI:92.3319, EMA1: 3.6401, EMA2: 3.6378
Fiyat:3.6394, Fark: 0.0014, RSI:92.7742, EMA1: 3.6408, EMA2: 3.6382
Fiyat:3.6395, Fark: 0.0004, RSI:91.6770, EMA1: 3.6392, EMA2: 3.6372
Fiyat:3.6382, Fark: -0.0010, RSI:90.6074, EMA1: 3.6380, EMA2: 3.6364
Fiyat:3.6382, Fark: 0.0000, RSI:90.8044, EMA1: 3.6382, EMA2: 3.6365
Fiyat:3.6364, Fark: -0.0002, RSI:88.3644, EMA1: 3.6362, EMA2: 3.6352
Fiyat:3.6406, Fark: 0.0007, RSI:93.0047, EMA1: 3.6412, EMA2: 3.6385
Fiyat:3.6410, Fark: 0.0001, RSI:92.9485, EMA1: 3.6411, EMA2: 3.6384
Fiyat:3.6390, Fark: 0.0011, RSI:92.5910, EMA1: 3.6405, EMA2: 3.6380
Fiyat:3.6390, Fark: 0.0020, RSI:76.9831, EMA1: 3.6393, EMA2: 3.6389
Fiyat:3.6399, Fark: -0.0015, RSI:68.3428, EMA1: 3.6384, EMA2: 3.6383
Fiyat:3.6401, Fark: -0.0017, RSI:93.0601, EMA1: 3.6409, EMA2: 3.6399
Fiyat:3.6387, Fark: 0.0054, RSI:95.3931, EMA1: 3.6441, EMA2: 3.6421
Fiyat:3.6437, Fark: 0.0020, RSI:96.0560, EMA

Fiyat:3.6126, Fark: -0.0037, RSI:4.0182, EMA1: 3.6089, EMA2: 3.6129
Fiyat:3.6126, Fark: -0.0033, RSI:4.1314, EMA1: 3.6093, EMA2: 3.6131
Fiyat:3.6126, Fark: -0.0003, RSI:6.0484, EMA1: 3.6138, EMA2: 3.6161
Fiyat:3.6123, Fark: 0.0040, RSI:7.4269, EMA1: 3.6156, EMA2: 3.6173
Fiyat:3.6152, Fark: -0.0010, RSI:6.3086, EMA1: 3.6142, EMA2: 3.6164
Fiyat:3.6152, Fark: -0.0031, RSI:4.3781, EMA1: 3.6121, EMA2: 3.6135
Fiyat:3.6107, Fark: 0.0002, RSI:3.7130, EMA1: 3.6109, EMA2: 3.6127
Fiyat:3.6116, Fark: -0.0010, RSI:3.5771, EMA1: 3.6106, EMA2: 3.6125
Fiyat:3.6116, Fark: -0.0025, RSI:3.1204, EMA1: 3.6094, EMA2: 3.6117
Fiyat:3.6116, Fark: 0.0001, RSI:4.1904, EMA1: 3.6118, EMA2: 3.6133
Fiyat:3.6123, Fark: 0.0002, RSI:4.1314, EMA1: 3.6117, EMA2: 3.6132
Fiyat:3.6123, Fark: -0.0003, RSI:4.3137, EMA1: 3.6120, EMA2: 3.6134
Döngü manuel olarak durduruldu.
